<a href="https://colab.research.google.com/github/PGLJIJI/Recommendation-System_Content-based_PGL/blob/main/Content_based_PGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 11:09:29--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  20.0MB/s    in 0.7s    

2022-01-02 11:09:30 (20.0 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2022-01-02 11:09:30--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.2MB/s    in 0.6s    

202

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [4]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [5]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]

ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

#產生舊戶名單
old_buyers = list(set(users) & set(ratings_trainings['reviewerID']))
#產生新戶名單
new_buyers = list(set(users) - set(ratings_trainings['reviewerID']))



In [6]:
ratings_trainings

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1500508800,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1489622400,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1488326400,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1487635200,2017-02-21



##資料擷取

In [7]:
# item 的評論數與平均評分
item_ratings = ratings_trainings[ratings_trainings['DATE'] >= '2017-09-01'].groupby('asin')[['overall']].mean().reset_index()
item_ratings.columns = ['asin', 'mean']
df_counts = pd.DataFrame(ratings_trainings.asin.value_counts()).reset_index()
df_counts.columns = ['asin','counts']
item_ratings = item_ratings.merge(df_counts)
item_ratings

,asin,mean,counts
0,0143026860,5.000000,17
1,014789302X,5.000000,20
2,0571348351,5.000000,3
3,0957664613,5.000000,1
4,0976070502,5.000000,1
...,...,...,...
8902,B01HIPOQ2M,3.600000,41
8903,B01HIUEEHO,5.000000,5
8904,B01HIWKGOM,5.000000,1
8905,B01HJ84SGM,5.000000,5


In [8]:
top10_rating_counts = item_ratings.sort_values(by='counts', ascending=False).asin.to_list()[0:10]
top10_rating_counts

['B000FOI48G',
 'B000GLRREU',
 '1620213982',
 'B001QY8QXM',
 'B01DKQAXC0',
 'B00W259T7G',
 'B006IB5T4W',
 'B00005JS5C',
 'B0012Y0ZG2',
 'B000WYJTZG']

In [9]:
#取出要用的特徵
metadata = metadata[['asin', 'title', 'description', 'rank', 'brand']]
metadata

,asin,title,description,rank,brand
0,6546546450,Loud 'N Clear&trade; Personal Sound Amplifier,[Loud 'N Clear Personal Sound Amplifier allows...,"2,938,573 in Beauty & Personal Care (",idea village
1,7178680776,No7 Lift &amp; Luminate Triple Action Serum 50...,[No7 Lift & Luminate Triple Action Serum 50ml ...,"872,854 in Beauty & Personal Care (",
2,7250468162,No7 Stay Perfect Foundation Cool Vanilla by No7,[No7 Stay Perfect Foundation now stays perfect...,"956,696 in Beauty & Personal Care (",No7
3,7367905066,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[],"1,870,258 in Beauty & Personal Care (",
4,7414204790,Lacto Calamine Skin Balance Oil control 120 ml...,[Lacto Calamine Skin Balance Daily Nourishing ...,"67,701 in Beauty & Personal Care (",Pirmal Healthcare
...,...,...,...,...,...
32887,B01HIWLLUK,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[],"2,145,325 in Beauty & Personal Care (",
32888,B01HJ1K3YK,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[],"1,639,713 in Beauty & Personal Care (",Salon Perfect
32889,B01HJ84SGM,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,[],"207,410 in Beauty & Personal Care (",
32890,B01HJASD20,12 White Feather Shuttlecocks Birdies Badminto...,[Brand new and high quality<br> Enables fast v...,"965,673 in Beauty & Personal Care (",GBSTORE


In [10]:
#從 rank 中取出 sub category
metadata['sub_category'] = metadata['rank'].str.split("in ").str[1].replace(r'\(','', regex = True).replace(r'&amp;','&', regex = True)
metadata['sub_category'].value_counts().rename_axis('sub_category').reset_index(name = 'counts')


,sub_category,counts
0,Beauty & Personal Care,32380
1,Grocery & Gourmet Food,38
2,Health & Household,21
3,Toys & Games,3
4,Sports & Outdoors,3
5,"Clothing, Shoes & Jewelry",2
6,Baby,2
7,Home & Kitchen,1
8,Tools & Home Improvement,1
9,Automotive,1


In [11]:
#刪除 rank 欄位
metadata = metadata.drop(columns = 'rank')
metadata

,asin,title,description,brand,sub_category
0,6546546450,Loud 'N Clear&trade; Personal Sound Amplifier,[Loud 'N Clear Personal Sound Amplifier allows...,idea village,Beauty & Personal Care
1,7178680776,No7 Lift &amp; Luminate Triple Action Serum 50...,[No7 Lift & Luminate Triple Action Serum 50ml ...,,Beauty & Personal Care
2,7250468162,No7 Stay Perfect Foundation Cool Vanilla by No7,[No7 Stay Perfect Foundation now stays perfect...,No7,Beauty & Personal Care
3,7367905066,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[],,Beauty & Personal Care
4,7414204790,Lacto Calamine Skin Balance Oil control 120 ml...,[Lacto Calamine Skin Balance Daily Nourishing ...,Pirmal Healthcare,Beauty & Personal Care
...,...,...,...,...,...
32887,B01HIWLLUK,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[],,Beauty & Personal Care
32888,B01HJ1K3YK,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[],Salon Perfect,Beauty & Personal Care
32889,B01HJ84SGM,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,[],,Beauty & Personal Care
32890,B01HJASD20,12 White Feather Shuttlecocks Birdies Badminto...,[Brand new and high quality<br> Enables fast v...,GBSTORE,Beauty & Personal Care


In [12]:
df_metadata_review = metadata.merge(item_ratings)
df_metadata_review


,asin,title,description,brand,sub_category,mean,counts
0,7178680776,No7 Lift &amp; Luminate Triple Action Serum 50...,[No7 Lift & Luminate Triple Action Serum 50ml ...,,Beauty & Personal Care,3.000000,1
1,7414204790,Lacto Calamine Skin Balance Oil control 120 ml...,[Lacto Calamine Skin Balance Daily Nourishing ...,Pirmal Healthcare,Beauty & Personal Care,3.666667,15
2,9790787006,Jenna Jameson Heartbreaker Perfume for women 3...,[Jenna Jameson Heartbreaker Perfume for women ...,Jenna Jameson,Beauty & Personal Care,3.666667,57
3,B000050B6B,Philips Norelco HQ5 Shaving Heads,[Replacement razor heads for Reflex Action raz...,Philips Norelco,Beauty & Personal Care,5.000000,154
4,B000050B6B,Philips Norelco HQ5 Shaving Heads,[Replacement razor heads for Reflex Action raz...,Philips Norelco,Beauty & Personal Care,5.000000,154
...,...,...,...,...,...,...,...
8956,B01HIPOQ2M,YSL YVES SAINT LAURENT Mon Paris 1.2ml/0.04oz ...,[Eau de parfum spray vial mini design house: y...,Yves Saint Laurent,Beauty & Personal Care,3.600000,41
8957,B01HIUEEHO,Pokemon Plush 9.2 Inch / 23cm Gengar Doll Stuf...,[Pokemon Plush 9.2 Inch / 23cm Gengar Doll Stu...,,Beauty & Personal Care,5.000000,5
8958,B01HIWKGOM,Benefit Cosmetics All Purpose Makeup Pencil Sh...,[New and unused product. 100% authentic Benefi...,Benefit Cosmetics,Beauty & Personal Care,5.000000,1
8959,B01HJ84SGM,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,[],,Beauty & Personal Care,5.000000,5


In [13]:
import numpy as np
# df_metadata_review = df_metadata_review.replace('', np.nan)
# df_metadata_review

In [14]:
df_user_subcat = ratings_trainings.merge(df_metadata_review[['asin', 'sub_category']]).groupby('reviewerID').agg({'sub_category': lambda x: x.tolist()}).reset_index()
df_user_subcat

,reviewerID,sub_category
0,A0010876CNE3ILIM9HV0,[Beauty & Personal Care ]
1,A001170867ZBE9FORRQL,"[Beauty & Personal Care , nan]"
2,A0028738FYF1SKPPC7B1,[Beauty & Personal Care ]
3,A0045356A23634W7RI4M,[Beauty & Personal Care ]
4,A006277084SDH2LTUV6X,[Beauty & Personal Care ]
...,...,...
238226,AZZVV7VDE6EF1,[Beauty & Personal Care ]
238227,AZZW8CF93X1TC,[Beauty & Personal Care ]
238228,AZZWNU4RB5BY1,[Beauty & Personal Care ]
238229,AZZXKG9AGRVGU,[Beauty & Personal Care ]


In [15]:
#將同 sub category 的 item 整理在一起
df_subcat_item = df_metadata_review[['sub_category', 'asin']].groupby('sub_category').agg({'asin': lambda x: x.tolist()}).reset_index()
df_subcat_item

,sub_category,asin
0,Automotive,[B01BHN3EHE]
1,Beauty & Personal Care,"[7178680776, 7414204790, 9790787006, B000050B6..."
2,"Clothing, Shoes & Jewelry","[B010B88DLS, B013NEZMZC]"
3,Grocery & Gourmet Food,"[B000FQMMY6, B000OQ4AO2, B0014ZA9QU, B0019J8Q5..."
4,Health & Household,"[B000ZMBSNG, B00I3F4QSQ, B00OK84FOA, B00UMR1C0..."
5,Home & Kitchen,[B0159F88UO]
6,Sports & Outdoors,"[B00RWW1Z2Q, B01CDUH3XG]"
7,Tools & Home Improvement,[B01DJOBF2U]
8,Toys & Games,[B01GI4AHC2]


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
subcat_dum = pd.get_dummies(df_metadata_review['sub_category'] , columns = ['subcat'])
subcat_dum

,Automotive,Beauty & Personal Care,"Clothing, Shoes & Jewelry",Grocery & Gourmet Food,Health & Household,Home & Kitchen,Sports & Outdoors,Tools & Home Improvement,Toys & Games
0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
8956,0,1,0,0,0,0,0,0,0
8957,0,1,0,0,0,0,0,0,0
8958,0,1,0,0,0,0,0,0,0
8959,0,1,0,0,0,0,0,0,0


In [19]:
df_user_brand = ratings_trainings.merge(df_metadata_review[['asin', 'brand']]).groupby('reviewerID').agg({'brand': lambda x: x.tolist()}).reset_index()
df_user_brand

,reviewerID,brand
0,A0010876CNE3ILIM9HV0,[illunt]
1,A001170867ZBE9FORRQL,"[Wazor, ]"
2,A0028738FYF1SKPPC7B1,[Indian Earth]
3,A0045356A23634W7RI4M,[GENERIC]
4,A006277084SDH2LTUV6X,[]
...,...,...
238226,AZZVV7VDE6EF1,[]
238227,AZZW8CF93X1TC,[Technic]
238228,AZZWNU4RB5BY1,[BAISIDAI]
238229,AZZXKG9AGRVGU,[]


In [20]:
#將同 brand 的 item 整理在一起
df_metadata_review['brand'] = df_metadata_review['brand'].str.replace('#','').replace('\(','',regex=True).replace('\)','',regex=True).replace(',','',regex=True).replace('-','',regex=True).replace('\.','',regex=True).replace('\'','',regex=True).replace('\*','',regex=True).replace('', np.nan)
df_brand_item = df_metadata_review[['brand', 'asin']].groupby('brand').agg({'asin': lambda x: x.tolist()}).reset_index()
df_brand_item

,brand,asin
0,100% AUTHENTIC Mfg only byTrue Tape LLC,[B00EBCS3TS]
1,111SKIN NAC Y2,[B019FCU3PU]
2,14 oz DabOn,[B001JKRNX2]
3,1820 Omega Visage,[B01FWDRJMQ]
4,1stopsalon,"[B007T79ARQ, B007T7AR3W]"
...,...,...
3234,willatram,"[B00COGVXNQ, B00FZM7KF6]"
3235,willy go wild,[B0108LZAOQ]
3236,younique,[B014JO6U1A]
3237,youthwaters,[B01H2DXMAC]


In [21]:
#將 brand 轉換成 one hot encoding
from sklearn.metrics.pairwise import cosine_similarity
brand_dum = pd.get_dummies(df_metadata_review['brand'] , columns = ['brand'])


In [ ]:
brand_dum

,100% AUTHENTIC Mfg only byTrue Tape LLC,111SKIN NAC Y2,14 oz DabOn,1820 Omega Visage,1stopsalon,3 Pack NYC Ultra Moist Lipwear Violet Shine,3 Way Poncho,350buy,3M,40 Carrots,47krate,4life transfer factor renewall,7 Miracles,7th Heaven,80 Acres,A LA MAISON,A Size Above,AAA DE AROMAS ARTESANALES DE ANTIGUA,AAATSSox,AB,ABCsell764,ABSOLUTE NEW YORK,AC,ACCmall,ACE,ACT,ACURE,ADC,AI Sports Nutrition,AJ Wholesale,AKOAK,ALAZCO,ALINCAS,ALIVER,AMBI,AMERICAN CREW,ANDREA HAIR GROWTH ESSENCE,AOFUL,API,APRIL SKIN,...,party supplies,peeninedeer market,personal care,qs jewelry,ranbaxy,redcolourful,reskin,richbest,rootsherbalcom,safeHands,sanofi,schwarzkopf,sd,selltop15,sensuva,shiseido,skncticls,stay on satin 779B black,sunkiller,superkrit,supertap,szh,theiss,thinkThin,thisworks,too cool for school,top apl,totes,uiphgjwexzv,unbrand,uxcell,value makers,vi derm,we,wet n wild,willatram,willy go wild,younique,youthwaters,zonman
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8956,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8957,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8958,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8959,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
# 把 description 從 list 轉為 str
df_metadata_review['description'] = df_metadata_review['description'].apply(lambda x: ' '.join(x))
type(df_metadata_review['description'][0])

str

In [23]:
#將 title 與 description 合併

df_metadata_review['title_description'] = df_metadata_review['title'] + df_metadata_review['description']
df_metadata_review['title_description'] = df_metadata_review['title_description'].str.lower()

## 產生推薦

In [24]:
import string
import re
import nltk
## remove punctuation
punctuation_table = str.maketrans('', '', string.punctuation)
df_metadata_review['title_description'] = df_metadata_review['title_description'].str.translate(punctuation_table)

In [25]:
df_metadata_review

,asin,title,description,brand,sub_category,mean,counts,title_description
0,7178680776,No7 Lift &amp; Luminate Triple Action Serum 50...,No7 Lift & Luminate Triple Action Serum 50ml b...,NaN,Beauty & Personal Care,3.000000,1,no7 lift amp luminate triple action serum 50ml...
1,7414204790,Lacto Calamine Skin Balance Oil control 120 ml...,Lacto Calamine Skin Balance Daily Nourishing L...,Pirmal Healthcare,Beauty & Personal Care,3.666667,15,lacto calamine skin balance oil control 120 ml...
2,9790787006,Jenna Jameson Heartbreaker Perfume for women 3...,Jenna Jameson Heartbreaker Perfume for women 3...,Jenna Jameson,Beauty & Personal Care,3.666667,57,jenna jameson heartbreaker perfume for women 3...
3,B000050B6B,Philips Norelco HQ5 Shaving Heads,Replacement razor heads for Reflex Action razo...,Philips Norelco,Beauty & Personal Care,5.000000,154,philips norelco hq5 shaving headsreplacement r...
4,B000050B6B,Philips Norelco HQ5 Shaving Heads,Replacement razor heads for Reflex Action razo...,Philips Norelco,Beauty & Personal Care,5.000000,154,philips norelco hq5 shaving headsreplacement r...
...,...,...,...,...,...,...,...,...
8956,B01HIPOQ2M,YSL YVES SAINT LAURENT Mon Paris 1.2ml/0.04oz ...,Eau de parfum spray vial mini design house: yv...,Yves Saint Laurent,Beauty & Personal Care,3.600000,41,ysl yves saint laurent mon paris 12ml004oz via...
8957,B01HIUEEHO,Pokemon Plush 9.2 Inch / 23cm Gengar Doll Stuf...,Pokemon Plush 9.2 Inch / 23cm Gengar Doll Stuf...,NaN,Beauty & Personal Care,5.000000,5,pokemon plush 92 inch 23cm gengar doll stuffe...
8958,B01HIWKGOM,Benefit Cosmetics All Purpose Makeup Pencil Sh...,New and unused product. 100% authentic Benefit...,Benefit Cosmetics,Beauty & Personal Care,5.000000,1,benefit cosmetics all purpose makeup pencil sh...
8959,B01HJ84SGM,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,,NaN,Beauty & Personal Care,5.000000,5,now dmannose 500 mg 120 veg capsules pack of 3


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算商品用標題所表示的 tfidf 矩陣
df = df_metadata_review.drop_duplicates('title_description')
tf = TfidfVectorizer(analyzer='word', stop_words='english', min_df=0.001)
tfidf_matrix = tf.fit_transform(df_metadata_review['title_description'])

In [27]:
tf.get_feature_names_out()

array(['005', '01', '011', ..., 'zippered', 'zone', 'zum'], dtype=object)

In [28]:
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
tfidf_df = pd.DataFrame(data=tfidf_matrix.toarray(), columns=tf.get_feature_names_out())

In [40]:
concated_tfidf_df = pd.concat([tfidf_df, subcat_dum, brand_dum], axis=1)
similarity_matrix

array([[1.        , 0.40824829, 0.40824829, ..., 0.40824829, 0.5       ,
        0.5       ],
       [0.40824829, 1.        , 0.33333333, ..., 0.33333333, 0.4871998 ,
        0.40824829],
       [0.40824829, 0.33333333, 1.        , ..., 0.33333333, 0.40824829,
        0.40824829],
       ...,
       [0.40824829, 0.33333333, 0.33333333, ..., 1.        , 0.40824829,
        0.40824829],
       [0.5       , 0.4871998 , 0.40824829, ..., 0.40824829, 1.        ,
        0.5       ],
       [0.5       , 0.40824829, 0.40824829, ..., 0.40824829, 0.5       ,
        1.        ]])

In [38]:

# 計算商品間的相似程度
similarity_matrix = cosine_similarity(concated_tfidf_df)
mapping = pd.Series(df_metadata_review.index,index = df_metadata_review['title_description'])

# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res







In [37]:
similarity_matrix.shape

(8961, 8961)

In [39]:
##針對舊客戶，基於title+description來做content-based推薦；針對新客戶，推薦其站上熱銷品(進一年評論數排名前十)
def recommender1(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    ratings_trainings = training_data

    for user in users:
      recom_list = recommend_items(df_metadata_review[df_metadata_review['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title_description'].tolist(), k)[:k]
      if recom_list:
        recommendations[user] = recom_list
      else:
        recommendations[user] = top10_rating_counts

    return recommendations


ratings_by_user = recommender1(ratings_trainings, users)
ratings_by_user

{'A100XQFWKQ30O2': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A103T1QOGFCSEH': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A106UKKSJ2KXPF': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A10A7GV4D5A11V': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A1119JJ37ZLB8R': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W259T7G',
  'B006IB5T4W',
  'B00005JS5C',
  'B0012Y0ZG2',
  'B000WYJTZG'],
 'A113UOOLBSZN52': ['B000FOI48G',
  'B000GLRREU',
  '1620213982',
  'B001QY8QXM',
  'B01DKQAXC0',
  'B00W

In [ ]:
# def recommender2(training_data, users=[], k=10):
#     '''
#     * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
#     * users: [] 需要被推薦的使用者
#     * k: int 每個使用者需要推薦的商品數
#     * recommendations: dict
#       {
#           使用者一： [推薦商品一, 推薦商品二, ...],
#           使用者二： [...], ...
#       }
#     '''
#     recommendations = {}
#     ratings_trainings = training_data

    

#     for user in users:
      
#       recom_list = recommend_items(df_metadata_review[df_metadata_review['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title_description'].tolist(), k)[:int(0.5*k)]
      
#       user_bought_brand_list = df_user_brand[df_user_brand['reviewerID'] == user]['brand'].tolist()
#       same_brand_item_list = df_brand_item[df_brand_item['brand'].isin(user_bought_brand_list)]['asin'].tolist()

#       user_bought_subcat_list = df_user_subcat[df_user_subcat['reviewerID'] == user]['sub_category'].tolist()
#       same_subcat_item_list = df_subcat_item[df_subcat_item['sub_category'].isin(user_bought_subcat_list)]['asin'].tolist()

#       if recom_list:
#         import random 
#         def select_candidate(item_df, condition_list,  count):
#           answer_all = []
#           for condition in condition_list:
#             answer = item_df[item_df['asin'].isin(condition)]['asin'].tolist()
#             if len(answer) >= count:
#               answer = random.choices(answer, k=count)

#             if len(answer_all) <= count:
#               unique_answer = list(set(answer))
#               answer_all.extend(unique_answer)
#             else:
#               return list(set(answer_all))[:count]

#           return list(set(answer_all))[:count]


#         candidate_list = [recom_list, same_subcat_item_list, same_brand_item_list, top10_rating_counts]
#         final_recom_list = select_candidate(item_df=df_metadata_review, condition_list=candidate_list, count=k)
        
#         recommendations[user] = final_recom_list
        
#       else:
#         recommendations[user] = top10_rating_counts

#     return recommendations


# ratings_by_user = recommender2(ratings_trainings, users)


## 結果評估

In [35]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.08305084745762711